In [1]:
import pandas as pd
import qgrid
import json
import os
import functools
import ipywidgets as widgets
from pathlib import Path
from IPython.display import clear_output, display
from ipyleaflet import Map, Icon, Marker, AntPath, basemaps, basemap_to_tiles, LayerException
from OSGridConverter import grid2latlong

import sys
sys.path.append('..')
from src import sandbox, gpx
from src.settings import PROJECT_PATH, DATASET_PATH

In [2]:
file_gen = DATASET_PATH.glob("*walks.json")

dfs = [] # an empty list to store the data frames
for file in file_gen:
    
    with open(file) as f:
        
        data = json.load(f)
        dfs.append(pd.DataFrame(data))

df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

display(df)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,...,Graham / Donald,Donald,Corbett / Donald,Grahams:,Graham:,Grahams,Sub 2000s,Donalds,Corbett:,Link
0,9.5km / 6 miles,3 - 3.5 hours,96m (Profile),NG277051,islands,canna-souterrains,The Canna Souterrains,3.00,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.5km / 2.75 miles,1 - 2 hours,46m (Profile),NG277051,islands,canna-explorer,Canna Explorer,4.00,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19.5km / 12 miles,NaN,581m (Profile),NG277051,islands,canna-coast,Canna Coastal circuit,5.00,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.25km / 7.5 miles,3.5 - 4 hours,65m (Profile),NG277051,islands,sanday-puffins,Isle of Sanday and the Puffins,4.00,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17km / 10.5 miles,NaN,892m (Profile),NH203394,lochness,an-sidhean,"An Sìdhean from Loch Monar, Glen Strathfarrar",4.00,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,11.5km / 7.25 miles,NaN,1326m (Profile),NG935566,torridon,Liathach,"Liathach, Glen Torridon",4.11,44,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Liath...
3810,12km / 7.5 miles,NaN,715m (Profile),NG798599,torridon,Diabaig,Diabaig Coastal Circuit,3.57,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Diaba...
3811,12km / 7.5 miles,NaN,980m (Profile),NG889542,torridon,Beinndamh,Beinn Dàmh (or Ben Damph),4.08,13,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Beinn...
3812,8km / 5 miles,NaN,350m (Profile),NG798599,torridon,Inveralligin,Diabaig to Inveralligin,4.00,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Inver...


We have two columns for time as some of the higher graded walks are given in summer conditions. Let's combine those.
Similary, some columns are plurals of others, let's combine those too.

In [3]:
df["Time"] = df["Time"].fillna(df["Time (summer conditions)"])
df = df.drop(columns="Time (summer conditions)")

for col in df.columns:
    if col + 's' in df.columns:
        df[col] = df[col].fillna(df[col + 's'])
        df = df.drop(columns=col + 's')

display(df)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,...,Corbett,Munro,Graham,Graham / Donald,Donald,Corbett / Donald,Grahams:,Graham:,Corbett:,Link
0,9.5km / 6 miles,3 - 3.5 hours,96m (Profile),NG277051,islands,canna-souterrains,The Canna Souterrains,3.00,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.5km / 2.75 miles,1 - 2 hours,46m (Profile),NG277051,islands,canna-explorer,Canna Explorer,4.00,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19.5km / 12 miles,8 - 9 hours,581m (Profile),NG277051,islands,canna-coast,Canna Coastal circuit,5.00,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.25km / 7.5 miles,3.5 - 4 hours,65m (Profile),NG277051,islands,sanday-puffins,Isle of Sanday and the Puffins,4.00,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17km / 10.5 miles,6 - 7 hours,892m (Profile),NH203394,lochness,an-sidhean,"An Sìdhean from Loch Monar, Glen Strathfarrar",4.00,1,4,...,An Sidhean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,11.5km / 7.25 miles,8 - 10 hours,1326m (Profile),NG935566,torridon,Liathach,"Liathach, Glen Torridon",4.11,44,5,...,NaN,"Spidean a' Choire Lèith, Mullach an Rathain",NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Liath...
3810,12km / 7.5 miles,4 - 5 hours,715m (Profile),NG798599,torridon,Diabaig,Diabaig Coastal Circuit,3.57,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Diaba...
3811,12km / 7.5 miles,5 - 6 hours,980m (Profile),NG889542,torridon,Beinndamh,Beinn Dàmh (or Ben Damph),4.08,13,3,...,Beinn Dàmh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Beinn...
3812,8km / 5 miles,2.5 - 3.5 hours,350m (Profile),NG798599,torridon,Inveralligin,Diabaig to Inveralligin,4.00,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.walkhighlands.co.uk/torridon/Inver...


Let's have a look at walks that include Munro's only. While we're at it, let's look at the really tough one's! We'll also drop columns that have all null values.

In [4]:
munro_walks = df[(df['Munro'].notnull()) & (df['Grade'].astype(int) >= 3)]
munro_walks = munro_walks.dropna(axis=1, how='all')
display(munro_walks)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,Bog,GPX,Corbett,Munro,Graham,Link
7,25km / 15.5 miles,7 - 10 hours,1567m (Profile),NH283386,lochness,strathfarrar-munros,Glen Strathfarrar Munros circuit,3.38,24,4,3,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Sgùrr na Ruaidhe, Càrn nan Gobhar, Sgùrr a' Ch...",NaN,NaN
10,28km / 17.5 miles,10 - 13 hours,1727m (Profile),NH216242,lochness,carn-eige,Càrn Eige and Mam Sodhail,3.73,11,4,3,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Càrn Eige (or Càrn Eighe), Beinn Fhionnlaidh, ...",NaN,NaN
11,16.5km / 10.25 miles,6 - 8 hours,1096m (Profile),NH216242,lochness,Tollcreagach,"Tom a' Chòinich and Toll Creagach, Glen Affric",3.65,17,4,3,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Toll Creagach, Tom a' Chòinich",NaN,NaN
12,29km / 18 miles,11 - 14 hours,1826m (Profile),NH218315,lochness,mullardoch-munros,The Loch Mullardoch Munros,3.42,12,4,4,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Càrn nan Gobhar, Sgùrr na Lapaich, An Riabhach...",NaN,NaN
22,20km / 12.5 miles,9 - 11 hours,1484m (Profile),NH079202,lochness,Ceathreamhnan,"Sgùrr nan Ceathreamhnan group, Alltbeithe",3.60,5,4,2,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Mullach nan Dheiragain, Sgùrr nan Ceathreamhna...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3776,29km / 18 miles,7 - 9 hours,990m (Profile),NN874662,perthshire,beinn-dearg,Beinn Dearg from near Blair Atholl,3.12,26,4,2,https://www.walkhighlands.co.uk/perthshire/pro...,NaN,Beinn Dearg,NaN,https://www.walkhighlands.co.uk/perthshire/bei...
3796,10.5km / 6.5 miles,7 - 8 hours,1190m (Profile),NG868576,torridon,Beinnalligin,Beinn Alligin,4.41,49,5,3,https://www.walkhighlands.co.uk/torridon/profi...,NaN,"Sgùrr Mòr, Tom na Gruagaich",NaN,https://www.walkhighlands.co.uk/torridon/Beinn...
3800,18km / 11.25 miles,7 - 9 hours,1116m (Profile),NG958569,torridon,Beinneighe,Beinn Eighe (western summits),4.23,30,4,2,https://www.walkhighlands.co.uk/torridon/profi...,NaN,"Ruadh-Stac Mòr, Spidean Coire nan Clach",NaN,https://www.walkhighlands.co.uk/torridon/Beinn...
3801,19km / 11.75 miles,8 - 9 hours,1144m (Profile),NH038624,torridon,Slioch,"Slioch, near Kinlochewe",4.04,25,4,2,https://www.walkhighlands.co.uk/torridon/profi...,NaN,Slioch,NaN,https://www.walkhighlands.co.uk/torridon/Slioc...


In [5]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

p = Map(
    basemap=basemap_to_tiles(basemaps.WaymarkedTrails.hiking),
    center=(57, 356),
    zoom=6
    )

#mark = Marker(location=[57, 356], icon=icon)
#m.add_layer(mark)

for p in munro_walks["Start Grid Ref"]:
    loc = grid2latlong(p)
    lat = loc.latitude
    lon = 360 + loc.longitude if loc.longitude < 0 else loc.longitude
    mark = Marker(location=[lat, lon])
    m.add_layer(mark)

m

Map(center=[57, 356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [8]:
df = sandbox.main()

widget = qgrid.show_grid(df)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
def get_lat_lon_bounds(df):
    
    north = max(df["lat"])
    south = min(df["lat"])
    east = max(df["lon"])
    west = min(df["lon"])
    
    return [(south, west), (north, east)]

def handle_click(map_handle, data, **kwargs):
    
    route = gpx.parse(data["GPX"])
    route = gpx.positive_long(route)
    latlon = list(zip(route.lat, route.lon))
    path = AntPath(locations=latlon)
    
    try:
        map_handle.substitute_layer(handle_click.current, path)
    except (AttributeError, LayerException):
        map_handle.add_layer(path)
        
    handle_click.current = path
    
    map_handle.center = kwargs["coordinates"]
    map_handle.zoom = 11
    # map_handle.fit_bounds = get_lat_lon_bounds(route)
   
    display(data)
    

In [8]:
filtered_df = widget.get_changed_df()

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

display_max = 100

if len(filtered_df.index) < display_max:
    
    for _, p in filtered_df.iterrows():
        mark = Marker(location=(p["lat"], p["lon"]), draggable=False, title=p["Name"])
        mark.on_click(functools.partial(handle_click, m, p))
        m.add_layer(mark)
        
else:
    print(f"Too many walks to show ({len(filtered_df.index)})! Filter below maximum ({display_max}) ")
    
m
        

NameError: name 'widget' is not defined